#### `:probabl.` Statistical ML Software Engineer Programming question

In [ ]:
import numpy as np
from sklearn.model_selection import permutation_test_score, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import load_iris

We will first load the iris dataset, including the labels (`y`), and split the data into training and test sets.

In [ ]:
X, y = load_iris(return_X_y=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=27)

In [ ]:
models = [DummyClassifier(), HistGradientBoostingClassifier()]

For each of the two models above, the loop below calculates (in cross-validated fashion with 5 folds):

* classification score — this is accuracy, for both models considered here
* permutation scores — these are classification scores (accuracy) using the same input data but with shuffled labels, in this case across the default of 100 permutations
* p-value — more on p-values below

In [ ]:
for model in models:
    score, permutation_score, p_value = permutation_test_score(
        model, X_train, y_train, random_state=27)
    print(f'For the {model} model we have:')
    print(f'    classification score: {score:.3f}')
    print(f'    permutation score (mean +/- standard deviation): {permutation_score.mean():.3f} +/- {permutation_score.std():.3f}')
    print(f'    p-value: {p_value:.3f}')
    print()

The name of the game with p-values and hypothesis testing is that we start by assuming there is no relationship between studied variables (or more precisely, that none is observed) — this is so called _null hypothesis_. We then calculate the _p-value_ to prove or — in most cases, generally try to — disprove that hypothesis. 

The p-value captures the probability of observed data happening by chance — p-value of 1 means that there is no observed relationship between our studied variables, we have a case of pure chance. Very low (close to 0) p-value means that the likelihood of the observed relationship being due to chance is very low, that there is some intrinsic relationship in the variables observed, or more precisely, that our model was able pick out this relationship.

What `permutation_test_score` does is calculate classification score across a number of permutations (in this case the default of 100) where the labels have been permuted/shuffled. The p-value is then calculated as _roughly_ the percentage of these permutations with scores better than the non-shuffled score — the exact formula is 

$$ \frac{C+1}{\text{n_permutations}+1} $$

where C is the number of permutations with score equal to or better than the non-shuffled score.

In case of the dummy model the (non-permuted) classification score is 0.358. That is because `DummyClassifier` does not consider the input features and by default predicts the most common class — in this case 0 — which in the train set occurs with 35.8\%.

In [ ]:
# This returns unique values in the numpy array and their counts
np.unique(y_train, return_counts=True)

In [ ]:
# This returns the percentage at which 
# the most common element in y_train occurs in the array
np.unique(y_train, return_counts=True)[1].max()/len(y_train)

For the dummy model the score of each of the 100 shuffled permutations is also 0.358 because the classifier predicts always the same most common label, resulting in the same accuracy. (That is also why we get the standard deviation of 0 for the permutation scores.) And by definition, since all 100 permutations have the same score, equal to the non-shuffled score, p-value is equal to 1.

The gradient boosting classifier gives accuracy of 95.8\%. Permutation scores are in line with dummy classifier scores (0.338 +/- 0.056). And because there are no permutation scores that are better than non-shuffled classification score, p-value is 0.01 (=1/101).

In conclusion, the histogram-based gradient boosting classifier results show that there is statistical relationship between iris features and its type — and that the model can detect the relationship. Results for the dummy classifier show that this particular model is unable to pick out this relationship (unsurprising given the nature of the dummy models in scikit-learn). 

(For the sake of completeness, I would at this point invoke the test dataset and calculate the _final_ score on the test set. Because feature engineering and cross validated hyperparameter optimization is out of scope of this exercise, a simple version of reporting the test set score would look something like this.)

In [ ]:
hgbc = HistGradientBoostingClassifier()

In [ ]:
hgbc.fit(X_train, y_train)

In [ ]:
hgbc.score(X_test, y_test)